## Get aligned MusicGen predictions

In [16]:
%cd /home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/

import numpy as np;
from embeddings.h5_processor import H5DataProcessor, DatasetConfig, ProcessedDataset, AudioSegmentSplitter;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split
import pandas as pd;
import yaml
import re

/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen


/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/.venv/lib64/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Use the existing H5 data processer infrastructure to get aligned audio data.

In [2]:
# Process datasets using H5DataProcessor
processor = H5DataProcessor()

with open("embeddings/config.yaml", 'r') as f:
    config = yaml.safe_load(f)

config = {
    'dataset': "embeddings/NHS",
    'subfolder': 's15-t15'
}

dataset = processor.process_h5_file(
    processor.get_embedding_path(DatasetConfig(**config)),
    DatasetConfig(**config)
)

In [ ]:
embeddings = dataset.embeddings
filenames = dataset.filenames
categories = pd.read_csv("data/NHS/categories.csv")

labels = [categories[categories['song'] == re.search(r'(NHSDiscography-\d+)_', filename).group(1) + '.wav']['type'].values for filename in filenames]

# Paste from classifier.py

In [27]:
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import yaml
from embeddings.h5_processor import H5DataProcessor, DatasetConfig, ProcessedDataset, AudioSegmentSplitter

In [42]:
with open("universal_music/config.yaml", 'r') as f:
    config = yaml.safe_load(f)

# Process datasets using H5DataProcessor
processor = H5DataProcessor()
all_train_data = []
all_test_data = []
class_names = set()

# Process each dataset and split
for dataset_config in config['datasets']:
    dataset = processor.process_h5_file(
        processor.get_embedding_path(DatasetConfig(**dataset_config)),
        DatasetConfig(**dataset_config)
    )
    
    # Split the dataset
    train_data, test_data = processor.get_train_test_split(
        dataset, 
        test_ratio=0.2,
        random_seed=42
    )
    
    all_train_data.append(train_data)
    all_test_data.append(test_data)
    class_names.update(dataset.labels)

# Combine datasets
combined_train = ProcessedDataset(
    embeddings=np.vstack([d.embeddings for d in all_train_data]),
    labels=[l for d in all_train_data for l in d.labels],
    filenames=[f for d in all_train_data for f in d.filenames],
    name="combined",
    num_samples=sum(d.num_samples for d in all_train_data)
)

combined_test = ProcessedDataset(
    embeddings=np.vstack([d.embeddings for d in all_test_data]),
    labels=[l for d in all_test_data for l in d.labels],
    filenames=[f for d in all_test_data for f in d.filenames],
    name="combined",
    num_samples=sum(d.num_samples for d in all_test_data)
)

# namespace! 
train_data = combined_train
test_data = combined_test

# Create and configure model
model = LogisticRegression(max_iter=1000)

In [43]:
"""Train and evaluate the classification model using pre-split data."""
# Create label mapping for string class labels
unique_labels = sorted(set(train_data.labels + test_data.labels))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

# Convert labels to indices
X_train = train_data.embeddings
y_train = np.array([label_to_idx[label] for label in train_data.labels])
X_test = test_data.embeddings
y_test = np.array([label_to_idx[label] for label in test_data.labels])


# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model.fit(X_train_scaled, y_train)

# Get predictions
y_pred = model.predict(X_test_scaled)

# Compute metrics
conf_matrix = confusion_matrix(y_test, y_pred)

TODO: figure out how to associate each of the predictions with its song ID
Then get the human ratings for only those songs, pick the 'winning' category, and create a CM